In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Data Downloading

In [2]:
# Team data
team_df = pd.read_csv('/Users/ansat.omurzakov/Desktop/TUe/Visualization/Ass1/FIFA DataSet/Data/FIFA World Cup 2022 Team Data/team_data.csv', delimiter=',')

# Match data
match_df = pd.read_csv('/Users/ansat.omurzakov/Desktop/TUe/Visualization/Ass1/FIFA DataSet/Data/FIFA World Cup 2022 Match Data/data.csv', delimiter=',')

# Historical data
df_historic_fifa_ranking = pd.read_csv('/Users/ansat.omurzakov/Desktop/TUe/Visualization/Ass1/FIFA DataSet/Data/FIFA World Cup Historic/fifa_ranking_2022-10-06.csv', delimiter=',')
df_historic_world_cup = pd.read_csv('/Users/ansat.omurzakov/Desktop/TUe/Visualization/Ass1/FIFA DataSet/Data/FIFA World Cup Historic/world_cup.csv', delimiter=',')

## Working with team data

In [3]:
dic2 = {}

dic = match_df[['home_team', 'home_manager']].to_dict()
for i in range(0, 64):
    team_name = dic['home_team'][i]
    team_manager = dic['home_manager'][i]
    if team_name == 'IR Iran':
        team_name = 'Iran'
    dic2[team_name] = team_manager

dic3 = match_df[['away_team', 'away_manager']].to_dict()
for i in range(0, 64):
    team_name = dic3['away_team'][i]
    team_manager = dic3['away_manager'][i]
    if team_name == 'IR Iran':
        team_name = 'Iran'
    dic2[team_name] = team_manager

team_df['manager'] = team_df['team'].apply(lambda x: dic2[x])

### Creation of Defensive and Attacking tackles columns

In [4]:
team_df['defensive_tackles_pct']= round(((team_df['tackles_def_3rd'] + team_df['tackles_mid_3rd']) / team_df['tackles']) * 100,1)
team_df['attacking_tackles_pct'] = round(((team_df['tackles_att_3rd']) / team_df['tackles']) * 100,1)

### Creation of Defensive and Attacking touch columns

In [5]:
team_df['touches_total'] = team_df['touches_def_pen_area'] + team_df['touches_def_3rd'] + team_df['touches_att_3rd'] + team_df['touches_mid_3rd'] + team_df['touches_att_pen_area']
team_df['defensive_ball_control_pct'] = round((team_df['touches_def_pen_area'] + team_df['touches_def_3rd'])/team_df['touches'] * 100, 1)
team_df['attacking_ball_control_pct'] = round((team_df['touches_mid_3rd'] + team_df['touches_att_3rd'] + team_df['touches_att_pen_area'])/team_df['touches'] * 100, 1)

### The percentage of own goals from goals conceded

In [6]:
team_df['goals_against_own_goals_pct'] = round(team_df['own_goals']/team_df['gk_goals_against'] * 100, 1)

### Balls recovered percentage

In [7]:
team_df['passes_not_received'] = team_df['passes'] - team_df['passes_received']
team_df['balls_recovered_pct'] = round((team_df['ball_recoveries'] + team_df['interceptions'])/ (team_df['passes_not_received'] + team_df['aerials_lost']+ team_df['miscontrols'] + team_df['dispossessed'])* 100,1)

### Creating statistics showing the progressive distance between countries

In [8]:
team_df['progressive_received_passes_pct'] = round(team_df['progressive_passes_received']/team_df['progressive_passes']*100, 1)
team_df['progressive_passes_pct'] = round(team_df['progressive_passes']/team_df['passes']*100, 1)
team_df['progressive_passes_distance_pct'] = round(team_df['passes_progressive_distance']/team_df['passes_total_distance']*100, 1)

### Attacking passes

In [9]:
team_df['attacking_passes_pct'] = round((team_df['passes_into_final_third'] + team_df['passes_into_penalty_area']) / team_df['passes_received']* 100, 1)

### Fouls to fouled ratio, to see which team committs more fouls than it fouled their opponents

In [10]:
team_df['fouls_ratio'] = team_df['fouls']/team_df['fouled']

### Assigning team to Defensive/Attacking/Overall poor play

In [11]:
team_df['xg_net'] = team_df['goals'] - round(team_df['xg'])
team_df['norm_goals_against'] = team_df['gk_goals_against'].apply(lambda x: round((x-5)/2.33184,2))

In [12]:
team_df[team_df['team'] == 'Morocco']

,team,players_used,avg_age,possession,games,games_starts,minutes,minutes_90s,goals,assists,...,attacking_ball_control_pct,goals_against_own_goals_pct,passes_not_received,balls_recovered_pct,progressive_received_passes_pct,progressive_passes_pct,progressive_passes_distance_pct,attacking_passes_pct,fouls_ratio,norm_goals_against
18,Morocco,25,27.2,39.0,7,77,660,7.3,6,4,...,61.4,20.0,653,47.7,97.4,4.0,39.2,6.3,1.185185,0.0


In [13]:
def labeling_team_strat(data):
    if data.loc['xg_net'] < 0 and data.loc['norm_goals_against'] > 0:
        return 'Bad attack; Bad defense'
    elif data.loc['xg_net'] < 0 and data.loc['norm_goals_against'] <= 0:
        return 'Bad attack; Good defense'
    elif data.loc['xg_net'] >= 0 and data.loc['norm_goals_against'] > 0:
        return 'Good attack; Bad defense'
    elif data.loc['xg_net'] >= 0 and data.loc['norm_goals_against'] <= 0:
        return 'Good attack; Good defense'

In [14]:
team_df['Team Strategy'] = team_df.apply(labeling_team_strat, axis = 1)

In [15]:
team_df[['team', 'goals', 'xg', 'xg_net', 'gk_goals_against', 'norm_goals_against', 'Team Strategy']]

,team,goals,xg,xg_net,gk_goals_against,norm_goals_against,Team Strategy
0,Argentina,15,15.2,0.0,8,1.29,Good attack; Bad defense
1,Australia,3,2.3,1.0,6,0.43,Good attack; Bad defense
2,Belgium,1,4.7,-4.0,2,-1.29,Bad attack; Good defense
3,Brazil,8,12.0,-4.0,3,-0.86,Bad attack; Good defense
4,Cameroon,4,3.4,1.0,4,-0.43,Good attack; Good defense
5,Canada,1,4.2,-3.0,7,0.86,Bad attack; Bad defense
6,Costa Rica,3,1.4,2.0,11,2.57,Good attack; Bad defense
7,Croatia,8,7.0,1.0,7,0.86,Good attack; Bad defense
8,Denmark,1,2.7,-2.0,3,-0.86,Bad attack; Good defense
9,Ecuador,4,3.7,0.0,3,-0.86,Good attack; Good defense


## Work on the radar chart

### Working on defense data

In [16]:
# So what we do is we normalize by min max scaling of all variables and assign weights to each

In [17]:
defensive_axis = team_df[['team', 'gk_save_pct', 'gk_clean_sheets_pct', 'gk_crosses_stopped_pct', 'gk_def_actions_outside_pen_area',
         'gk_avg_distance_def_actions', 'defensive_tackles_pct', 'dribble_tackles_pct', 'blocks', 'clearances', 'defensive_ball_control_pct', 
         'own_goals', 'gk_corner_kick_goals_against', 'gk_free_kick_goals_against', 'gk_pens_saved', 'gk_psxg_net']]
defensive_axis.describe()

,gk_save_pct,gk_clean_sheets_pct,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_avg_distance_def_actions,defensive_tackles_pct,dribble_tackles_pct,blocks,clearances,defensive_ball_control_pct,own_goals,gk_corner_kick_goals_against,gk_free_kick_goals_against,gk_pens_saved,gk_psxg_net
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.00000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,67.696875,29.775000,5.612500,3.812500,13.390625,89.078125,54.293750,44.843750,75.093750,42.69375,0.062500,0.406250,0.062500,0.156250,0.078125
std,12.387311,22.862605,4.629412,3.146298,3.644825,4.851986,9.811939,16.871639,31.511759,8.98328,0.245935,0.559918,0.245935,0.447889,1.486224
min,50.000000,0.000000,0.000000,0.000000,6.500000,76.100000,33.300000,21.000000,24.000000,23.50000,0.000000,0.000000,0.000000,0.000000,-2.700000
25%,56.450000,0.000000,2.875000,2.000000,10.950000,86.700000,47.400000,33.750000,52.750000,35.52500,0.000000,0.000000,0.000000,0.000000,-1.175000
50%,66.700000,33.300000,4.950000,3.000000,13.000000,89.750000,56.200000,41.000000,66.500000,41.55000,0.000000,0.000000,0.000000,0.000000,-0.050000
75%,79.325000,50.000000,7.500000,5.250000,15.550000,92.600000,59.625000,50.500000,96.250000,49.47500,0.000000,1.000000,0.000000,0.000000,1.075000
max,90.000000,66.700000,18.200000,14.000000,20.300000,95.800000,78.600000,86.000000,163.000000,65.20000,1.000000,2.000000,1.000000,2.000000,3.500000


In [18]:
for i in defensive_axis.columns:
    if 'pct' in i:
        defensive_axis[i]= round(defensive_axis[i]/100,2)

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/602362411.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defensive_axis[i]= round(defensive_axis[i]/100,2)


In [19]:
def min_max_scaling(df, column_name):
    min_value = df[column_name].min()
    max_value = df[column_name].max()
    df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)

In [20]:
lst1 = []
for i in defensive_axis.columns:
    if 'pct' not in i and i != 'team':
        lst1.append(i)

for team in lst1:
    min_max_scaling(defensive_axis, team)

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingW

In [21]:
defensive_axis

,team,gk_save_pct,gk_clean_sheets_pct,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_avg_distance_def_actions,defensive_tackles_pct,dribble_tackles_pct,blocks,clearances,defensive_ball_control_pct,own_goals,gk_corner_kick_goals_against,gk_free_kick_goals_against,gk_pens_saved,gk_psxg_net
0,Argentina,0.54,0.43,0.14,0.29,0.41,0.90,0.59,0.97,0.72,0.33,1.0,0.0,0.0,0.0,0.18
1,Australia,0.67,0.50,0.05,0.43,0.65,0.95,0.59,0.66,0.58,0.55,0.0,0.5,0.0,0.0,0.35
2,Belgium,0.82,0.67,0.03,0.14,0.54,0.96,0.63,0.18,0.25,0.50,0.0,0.0,0.0,0.5,0.77
3,Brazil,0.70,0.40,0.00,0.64,1.00,0.76,0.59,0.60,0.28,0.32,0.0,0.0,0.0,0.0,0.45
4,Cameroon,0.73,0.33,0.05,0.00,0.17,0.95,0.44,0.02,0.34,0.53,0.0,0.0,0.0,0.0,0.58
5,Canada,0.53,0.00,0.00,0.14,0.62,0.83,0.50,0.08,0.12,0.36,0.0,0.0,0.0,0.0,0.18
6,Costa Rica,0.55,0.33,0.04,0.14,0.25,0.94,0.51,0.32,0.42,0.65,0.0,0.0,0.0,0.0,0.21
7,Croatia,0.81,0.29,0.08,0.21,0.31,0.87,0.55,0.86,0.86,0.39,0.0,0.5,0.0,0.0,1.00
8,Denmark,0.73,0.33,0.00,0.36,0.92,0.90,0.46,0.34,0.18,0.33,0.0,0.0,0.0,0.0,0.55
9,Ecuador,0.50,0.33,0.04,0.14,0.47,0.96,0.33,0.18,0.13,0.40,0.0,0.0,0.0,0.0,0.29


In [22]:
defensive_axis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   team                             32 non-null     object 
 1   gk_save_pct                      32 non-null     float64
 2   gk_clean_sheets_pct              32 non-null     float64
 3   gk_crosses_stopped_pct           32 non-null     float64
 4   gk_def_actions_outside_pen_area  32 non-null     float64
 5   gk_avg_distance_def_actions      32 non-null     float64
 6   defensive_tackles_pct            32 non-null     float64
 7   dribble_tackles_pct              32 non-null     float64
 8   blocks                           32 non-null     float64
 9   clearances                       32 non-null     float64
 10  defensive_ball_control_pct       32 non-null     float64
 11  own_goals                        32 non-null     float64
 12  gk_corner_kick_goals_aga

In [23]:
defensive_axis.columns

Index(['team', 'gk_save_pct', 'gk_clean_sheets_pct', 'gk_crosses_stopped_pct',
       'gk_def_actions_outside_pen_area', 'gk_avg_distance_def_actions',
       'defensive_tackles_pct', 'dribble_tackles_pct', 'blocks', 'clearances',
       'defensive_ball_control_pct', 'own_goals',
       'gk_corner_kick_goals_against', 'gk_free_kick_goals_against',
       'gk_pens_saved', 'gk_psxg_net'],
      dtype='object')

In [24]:
defensive_axis['defensive_score'] = round(defensive_axis['gk_save_pct'] * 0.10 +
                                          defensive_axis['gk_clean_sheets_pct'] * 0.10 +
                                          defensive_axis['gk_crosses_stopped_pct'] * 0.07 +
                                          defensive_axis['gk_def_actions_outside_pen_area'] * 0.08 +
                                          defensive_axis['defensive_tackles_pct'] * 0.07 +
                                          defensive_axis['dribble_tackles_pct'] * 0.06 +
                                          defensive_axis['blocks'] * 0.06 +
                                          defensive_axis['clearances'] * 0.07 + 
                                          defensive_axis['defensive_ball_control_pct'] * 0.06 - 
                                          defensive_axis['own_goals'] * 0.06 - 
                                          defensive_axis['gk_corner_kick_goals_against'] * 0.05 - 
                                          defensive_axis['gk_free_kick_goals_against'] * 0.05 + 
                                          defensive_axis['gk_pens_saved'] * 0.05+
                                          defensive_axis['gk_psxg_net'] * 0.07 +
                                          defensive_axis['gk_avg_distance_def_actions'] * 0.07 , 2)

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3169887594.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defensive_axis['defensive_score'] = round(defensive_axis['gk_save_pct'] * 0.10 +


In [25]:
defensive_axis[defensive_axis['team'].isin(['France', 'Poland'])]

,team,gk_save_pct,gk_clean_sheets_pct,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_avg_distance_def_actions,defensive_tackles_pct,dribble_tackles_pct,blocks,clearances,defensive_ball_control_pct,own_goals,gk_corner_kick_goals_against,gk_free_kick_goals_against,gk_pens_saved,gk_psxg_net,defensive_score
11,France,0.82,0.14,0.09,0.36,0.51,0.90,0.6,0.77,0.57,0.35,0.0,0.5,0.0,0.0,0.65,0.39
20,Poland,0.82,0.50,0.02,0.00,0.10,0.86,0.4,0.28,0.62,0.51,0.0,0.0,0.0,1.0,0.89,0.43


In [26]:
min_max_scaling(defensive_axis, 'defensive_score')

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)


In [27]:
defensive_axis

,team,gk_save_pct,gk_clean_sheets_pct,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_avg_distance_def_actions,defensive_tackles_pct,dribble_tackles_pct,blocks,clearances,defensive_ball_control_pct,own_goals,gk_corner_kick_goals_against,gk_free_kick_goals_against,gk_pens_saved,gk_psxg_net,defensive_score
0,Argentina,0.54,0.43,0.14,0.29,0.41,0.90,0.59,0.97,0.72,0.33,1.0,0.0,0.0,0.0,0.18,0.56
1,Australia,0.67,0.50,0.05,0.43,0.65,0.95,0.59,0.66,0.58,0.55,0.0,0.5,0.0,0.0,0.35,0.88
2,Belgium,0.82,0.67,0.03,0.14,0.54,0.96,0.63,0.18,0.25,0.50,0.0,0.0,0.0,0.5,0.77,0.96
3,Brazil,0.70,0.40,0.00,0.64,1.00,0.76,0.59,0.60,0.28,0.32,0.0,0.0,0.0,0.0,0.45,0.92
4,Cameroon,0.73,0.33,0.05,0.00,0.17,0.95,0.44,0.02,0.34,0.53,0.0,0.0,0.0,0.0,0.58,0.44
5,Canada,0.53,0.00,0.00,0.14,0.62,0.83,0.50,0.08,0.12,0.36,0.0,0.0,0.0,0.0,0.18,0.16
6,Costa Rica,0.55,0.33,0.04,0.14,0.25,0.94,0.51,0.32,0.42,0.65,0.0,0.0,0.0,0.0,0.21,0.48
7,Croatia,0.81,0.29,0.08,0.21,0.31,0.87,0.55,0.86,0.86,0.39,0.0,0.5,0.0,0.0,1.00,0.92
8,Denmark,0.73,0.33,0.00,0.36,0.92,0.90,0.46,0.34,0.18,0.33,0.0,0.0,0.0,0.0,0.55,0.72
9,Ecuador,0.50,0.33,0.04,0.14,0.47,0.96,0.33,0.18,0.13,0.40,0.0,0.0,0.0,0.0,0.29,0.32


### Working on Attacking axis

In [28]:
attacking_axis = team_df[list(set(['team', 'shots_on_target_pct', 'average_shot_distance', 'sca', 'gca', 'attacking_tackles_pct', 'attacking_ball_control_pct',
                          'progressive_received_passes_pct', 'attacking_passes_pct', 'progressive_passes_pct', 'assists', 'average_shot_distance',
                          'crosses', 'goals', 'goals_per_shot', 'corner_kicks', 'dribbles_completed', 'miscontrols', 'errors', 'dispossessed', 'fouled']))]
attacking_axis.describe()

,progressive_received_passes_pct,crosses,attacking_ball_control_pct,attacking_passes_pct,dispossessed,attacking_tackles_pct,goals,errors,assists,miscontrols,shots_on_target_pct,goals_per_shot,gca,average_shot_distance,corner_kicks,fouled,progressive_passes_pct,dribbles_completed,sca
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.00000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,97.084375,68.031250,71.371875,8.965625,36.968750,10.921875,5.312500,1.312500,3.781250,61.843750,32.78750,0.105938,9.125000,17.909375,17.843750,47.562500,5.034375,23.062500,79.687500
std,1.369980,27.975201,6.518571,1.661637,15.476276,4.851986,4.130434,1.029798,3.220242,21.043901,9.42631,0.062002,7.329789,1.678202,9.618897,21.409882,0.909665,13.678185,41.545496
min,93.500000,21.000000,54.500000,5.200000,19.000000,4.200000,1.000000,0.000000,0.000000,38.000000,11.80000,0.000000,1.000000,15.500000,1.000000,23.000000,3.300000,8.000000,20.000000
25%,96.175000,52.000000,66.500000,7.775000,25.000000,7.400000,2.750000,1.000000,1.000000,47.000000,28.05000,0.067500,3.750000,16.575000,10.500000,34.750000,4.650000,13.000000,50.750000
50%,97.350000,59.000000,72.300000,9.050000,30.500000,10.250000,4.500000,1.000000,3.000000,56.000000,32.10000,0.100000,8.000000,18.100000,18.000000,38.000000,5.050000,19.000000,64.500000
75%,98.025000,79.750000,76.400000,10.500000,44.250000,13.300000,6.500000,2.000000,5.000000,70.250000,36.15000,0.140000,12.250000,18.775000,22.500000,51.750000,5.525000,28.000000,96.500000
max,99.100000,153.000000,84.200000,11.800000,81.000000,23.900000,16.000000,3.000000,12.000000,126.000000,58.30000,0.250000,29.000000,21.900000,39.000000,115.000000,6.800000,69.000000,183.000000


In [29]:
for i in attacking_axis.columns:
    if 'pct' in i:
        attacking_axis[i]= round(attacking_axis[i]/100,2)

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/1014135505.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  attacking_axis[i]= round(attacking_axis[i]/100,2)


In [30]:
lst2 = []
for i in attacking_axis.columns:
    if 'pct' not in i and i != 'team':
        lst2.append(i)

for team in lst2:
    min_max_scaling(attacking_axis, team)

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingW

In [31]:
attacking_axis['attacking_score'] = round( 
    0.10 * attacking_axis['shots_on_target_pct'] -
    0.07 * attacking_axis['average_shot_distance'] +
    0.08 * attacking_axis['sca'] +
    0.08 * attacking_axis['gca'] +
    0.06 * attacking_axis['attacking_tackles_pct'] +
    0.06 * attacking_axis['attacking_ball_control_pct'] +
    0.07 * attacking_axis['progressive_received_passes_pct'] +
    0.07 * attacking_axis['attacking_passes_pct'] +
    0.07 * attacking_axis['progressive_passes_pct'] +
    0.07 * attacking_axis['assists'] +
    0.06 * attacking_axis['crosses'] +
    0.09 * attacking_axis['goals'] +
    0.07 * attacking_axis['goals_per_shot'] +
    0.05 * attacking_axis['corner_kicks'] +
    0.06 * attacking_axis['dribbles_completed'] -
    0.05 * attacking_axis['miscontrols'] -
    0.05 * attacking_axis['errors'] -
    0.06 * attacking_axis['dispossessed'] +
    0.06 * attacking_axis['fouled']
    ,2)

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/2966530005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  attacking_axis['attacking_score'] = round(


In [32]:
attacking_axis[['team', 'attacking_score']].sort_values(by = 'attacking_score', ascending = False)

,team,attacking_score
11,France,0.62
10,England,0.57
0,Argentina,0.54
21,Portugal,0.44
3,Brazil,0.43
7,Croatia,0.43
19,Netherlands,0.41
26,Spain,0.33
12,Germany,0.32
16,Korea Republic,0.31


In [33]:
min_max_scaling(attacking_axis, 'attacking_score')

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)


In [34]:
attacking_axis

,progressive_received_passes_pct,crosses,attacking_ball_control_pct,attacking_passes_pct,dispossessed,attacking_tackles_pct,goals,errors,team,assists,...,shots_on_target_pct,goals_per_shot,gca,average_shot_distance,corner_kicks,fouled,progressive_passes_pct,dribbles_completed,sca,attacking_score
0,0.97,0.55,0.77,0.08,1.00,0.10,0.93,0.00,Argentina,0.67,...,0.43,0.48,0.86,0.44,1.00,1.00,0.05,0.61,0.98,0.85
1,0.96,0.25,0.62,0.07,0.27,0.05,0.13,1.00,Australia,0.25,...,0.31,0.48,0.18,0.47,0.18,0.12,0.04,0.08,0.13,0.15
2,0.96,0.24,0.65,0.08,0.08,0.04,0.00,0.33,Belgium,0.08,...,0.26,0.12,0.04,0.02,0.42,0.13,0.05,0.18,0.28,0.24
3,0.98,0.64,0.80,0.10,0.63,0.24,0.47,0.33,Brazil,0.50,...,0.42,0.28,0.43,0.31,0.95,0.55,0.06,0.39,0.92,0.65
4,0.94,0.24,0.66,0.11,0.08,0.05,0.20,0.33,Cameroon,0.33,...,0.57,0.56,0.21,0.14,0.29,0.16,0.05,0.26,0.18,0.43
5,0.99,0.25,0.76,0.10,0.18,0.17,0.00,0.67,Canada,0.08,...,0.12,0.12,0.04,0.23,0.29,0.15,0.05,0.38,0.25,0.19
6,0.95,0.00,0.55,0.05,0.19,0.06,0.13,0.33,Costa Rica,0.08,...,0.58,1.00,0.07,0.23,0.00,0.15,0.04,0.15,0.00,0.28
7,0.98,1.00,0.75,0.08,0.82,0.13,0.47,0.00,Croatia,0.67,...,0.33,0.40,0.46,0.47,0.76,0.73,0.05,0.52,0.69,0.65
8,0.96,0.35,0.78,0.11,0.10,0.10,0.00,0.00,Denmark,0.08,...,0.29,0.12,0.00,0.25,0.53,0.00,0.07,0.03,0.23,0.24
9,0.98,0.27,0.71,0.07,0.10,0.04,0.20,0.33,Ecuador,0.17,...,0.34,0.40,0.18,0.42,0.26,0.15,0.04,0.08,0.17,0.26


### Working with passes axis

In [35]:
passes_axis = team_df[['team', 'passes', 'progressive_passes_pct', 'passes_completed',
                        'pass_xa', 'assisted_shots', 'passes_offsides', 'passes_blocked']]
passes_axis.describe()

,passes,progressive_passes_pct,passes_completed,pass_xa,assisted_shots,passes_offsides,passes_blocked
count,32.000000,32.000000,32.000000,32.000000,32.00000,32.000000,32.000000
mean,2152.468750,5.034375,1746.562500,3.384375,33.59375,7.937500,37.937500
std,1024.281778,0.909665,913.510843,2.139959,17.88829,5.161505,13.795599
min,1101.000000,3.300000,769.000000,0.400000,7.00000,1.000000,22.000000
25%,1439.250000,4.650000,1121.000000,1.800000,21.75000,4.000000,29.750000
50%,1723.000000,5.050000,1317.000000,2.800000,27.00000,7.000000,35.500000
75%,2915.000000,5.525000,2320.500000,3.800000,38.00000,11.000000,42.250000
max,4625.000000,6.800000,3911.000000,9.200000,80.00000,23.000000,85.000000


In [36]:
for i in passes_axis.columns:
    if i == 'pass_xa':
        passes_axis[i] = round(passes_axis[i]/10,2)
    elif 'pct' in i:
        passes_axis[i]= round(passes_axis[i]/100,2)

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/2448464368.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_axis[i]= round(passes_axis[i]/100,2)
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/2448464368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_axis[i] = round(passes_axis[i]/10,2)


In [37]:
lst3 = []
for i in passes_axis.columns.copy():
    if 'pct' not in i and i != 'team':
        lst3.append(i)

print(lst3)
for team in lst3:
    min_max_scaling(passes_axis, team)

['passes', 'passes_completed', 'pass_xa', 'assisted_shots', 'passes_offsides', 'passes_blocked']


/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingW

In [38]:
passes_axis

,team,passes,progressive_passes_pct,passes_completed,pass_xa,assisted_shots,passes_offsides,passes_blocked
0,Argentina,1.00,0.05,1.00,0.76,0.95,1.00,0.63
1,Australia,0.17,0.04,0.15,0.08,0.14,0.00,0.30
2,Belgium,0.22,0.05,0.26,0.28,0.30,0.09,0.19
3,Brazil,0.60,0.06,0.63,0.85,0.86,0.32,0.29
4,Cameroon,0.04,0.05,0.06,0.20,0.19,0.05,0.03
5,Canada,0.14,0.05,0.17,0.25,0.27,0.32,0.14
6,Costa Rica,0.00,0.04,0.02,0.00,0.00,0.55,0.00
7,Croatia,0.97,0.05,0.95,0.64,0.74,0.45,1.00
8,Denmark,0.24,0.07,0.26,0.16,0.27,0.18,0.08
9,Ecuador,0.11,0.04,0.12,0.14,0.18,0.32,0.02


In [39]:
passes_axis['passes_score'] = round(0.10 * passes_axis['passes']+
                                    0.08 * passes_axis['progressive_passes_pct']+
                                    0.10 * passes_axis['passes_completed'] +
                                    0.08 * passes_axis['pass_xa'] +
                                    0.09 * passes_axis['assisted_shots'] - 
                                    0.05 * passes_axis['passes_offsides'] -
                                    0.05 * passes_axis['passes_blocked'],2)

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/552551553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_axis['passes_score'] = round(0.10 * passes_axis['passes']+


In [40]:
def min_0(value):
    if value < 0:
        return 0
    return value
passes_axis['passes_score'] = passes_axis['passes_score'].apply(min_0)
passes_axis

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/803993168.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_axis['passes_score'] = passes_axis['passes_score'].apply(min_0)


,team,passes,progressive_passes_pct,passes_completed,pass_xa,assisted_shots,passes_offsides,passes_blocked,passes_score
0,Argentina,1.00,0.05,1.00,0.76,0.95,1.00,0.63,0.27
1,Australia,0.17,0.04,0.15,0.08,0.14,0.00,0.30,0.04
2,Belgium,0.22,0.05,0.26,0.28,0.30,0.09,0.19,0.09
3,Brazil,0.60,0.06,0.63,0.85,0.86,0.32,0.29,0.24
4,Cameroon,0.04,0.05,0.06,0.20,0.19,0.05,0.03,0.04
5,Canada,0.14,0.05,0.17,0.25,0.27,0.32,0.14,0.06
6,Costa Rica,0.00,0.04,0.02,0.00,0.00,0.55,0.00,0.00
7,Croatia,0.97,0.05,0.95,0.64,0.74,0.45,1.00,0.24
8,Denmark,0.24,0.07,0.26,0.16,0.27,0.18,0.08,0.08
9,Ecuador,0.11,0.04,0.12,0.14,0.18,0.32,0.02,0.04


In [41]:
min_max_scaling(passes_axis, 'passes_score')

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)


In [42]:
passes_axis

,team,passes,progressive_passes_pct,passes_completed,pass_xa,assisted_shots,passes_offsides,passes_blocked,passes_score
0,Argentina,1.00,0.05,1.00,0.76,0.95,1.00,0.63,1.00
1,Australia,0.17,0.04,0.15,0.08,0.14,0.00,0.30,0.15
2,Belgium,0.22,0.05,0.26,0.28,0.30,0.09,0.19,0.33
3,Brazil,0.60,0.06,0.63,0.85,0.86,0.32,0.29,0.89
4,Cameroon,0.04,0.05,0.06,0.20,0.19,0.05,0.03,0.15
5,Canada,0.14,0.05,0.17,0.25,0.27,0.32,0.14,0.22
6,Costa Rica,0.00,0.04,0.02,0.00,0.00,0.55,0.00,0.00
7,Croatia,0.97,0.05,0.95,0.64,0.74,0.45,1.00,0.89
8,Denmark,0.24,0.07,0.26,0.16,0.27,0.18,0.08,0.30
9,Ecuador,0.11,0.04,0.12,0.14,0.18,0.32,0.02,0.15


### Working with possession axis

In [43]:
possession_axis = team_df[['team','possession', 'balls_recovered_pct', 'errors', 'miscontrols', 'dispossessed', 'tackles_won', 'interceptions', 'aerials_won_pct', 'passes_completed', ]]
possession_axis.describe()

,possession,balls_recovered_pct,errors,miscontrols,dispossessed,tackles_won,interceptions,aerials_won_pct,passes_completed
count,32.000000,32.00000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,49.443750,40.16875,1.312500,61.843750,36.968750,37.468750,33.375000,50.156250,1746.562500
std,9.457066,3.60881,1.029798,21.043901,15.476276,17.964094,14.282292,4.599364,913.510843
min,31.300000,33.50000,0.000000,38.000000,19.000000,20.000000,17.000000,36.700000,769.000000
25%,42.750000,37.57500,1.000000,47.000000,25.000000,27.000000,23.750000,47.825000,1121.000000
50%,50.150000,40.00000,1.000000,56.000000,30.500000,31.500000,31.000000,50.250000,1317.000000
75%,54.775000,41.85000,2.000000,70.250000,44.250000,39.500000,40.000000,53.150000,2320.500000
max,75.800000,47.90000,3.000000,126.000000,81.000000,88.000000,77.000000,58.800000,3911.000000


In [44]:
for i in possession_axis.columns:
    if 'pct' in i:
        possession_axis[i] = round(possession_axis[i]/100,2)

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/980690596.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  possession_axis[i] = round(possession_axis[i]/100,2)


In [45]:
lst4 = []
for i in possession_axis.columns.copy():
    if 'pct' not in i and i != 'team':
        lst4.append(i)

print(lst4)
for team in lst4:
    min_max_scaling(possession_axis, team)

['possession', 'errors', 'miscontrols', 'dispossessed', 'tackles_won', 'interceptions', 'passes_completed']


/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingW

In [46]:
possession_axis['possession_score'] = round(0.12 * possession_axis['possession'] +
                                            0.10 * possession_axis['balls_recovered_pct'] - 
                                            0.06 * possession_axis['errors'] -
                                            0.06 * possession_axis['miscontrols'] -
                                            0.07 * possession_axis['dispossessed'] +
                                            0.09 * possession_axis['tackles_won']+
                                            0.08 * possession_axis['interceptions'] +
                                            0.04 * possession_axis['aerials_won_pct'],2)

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3182653044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  possession_axis['possession_score'] = round(0.12 * possession_axis['possession'] +


In [47]:
possession_axis

,team,possession,balls_recovered_pct,errors,miscontrols,dispossessed,tackles_won,interceptions,aerials_won_pct,passes_completed,possession_score
0,Argentina,0.59,0.40,0.00,0.69,1.00,0.72,0.58,0.48,1.00,0.13
1,Australia,0.15,0.38,1.00,0.26,0.27,0.15,0.38,0.50,0.15,0.03
2,Belgium,0.58,0.37,0.33,0.10,0.08,0.10,0.00,0.54,0.26,0.11
3,Brazil,0.56,0.45,0.33,0.57,0.63,0.46,0.38,0.43,0.63,0.10
4,Cameroon,0.23,0.45,0.33,0.01,0.08,0.00,0.38,0.54,0.06,0.10
5,Canada,0.47,0.41,0.67,0.16,0.18,0.04,0.02,0.59,0.17,0.06
6,Costa Rica,0.00,0.44,0.33,0.06,0.19,0.22,0.23,0.56,0.02,0.07
7,Croatia,0.52,0.44,0.00,0.77,0.82,0.93,0.60,0.50,0.95,0.15
8,Denmark,0.64,0.39,0.00,0.14,0.10,0.10,0.10,0.52,0.26,0.14
9,Ecuador,0.49,0.35,0.33,0.31,0.10,0.15,0.18,0.50,0.12,0.10


In [48]:
min_max_scaling(possession_axis, 'possession_score')

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)


In [49]:
possession_axis

,team,possession,balls_recovered_pct,errors,miscontrols,dispossessed,tackles_won,interceptions,aerials_won_pct,passes_completed,possession_score
0,Argentina,0.59,0.40,0.00,0.69,1.00,0.72,0.58,0.48,1.00,0.67
1,Australia,0.15,0.38,1.00,0.26,0.27,0.15,0.38,0.50,0.15,0.00
2,Belgium,0.58,0.37,0.33,0.10,0.08,0.10,0.00,0.54,0.26,0.53
3,Brazil,0.56,0.45,0.33,0.57,0.63,0.46,0.38,0.43,0.63,0.47
4,Cameroon,0.23,0.45,0.33,0.01,0.08,0.00,0.38,0.54,0.06,0.47
5,Canada,0.47,0.41,0.67,0.16,0.18,0.04,0.02,0.59,0.17,0.20
6,Costa Rica,0.00,0.44,0.33,0.06,0.19,0.22,0.23,0.56,0.02,0.27
7,Croatia,0.52,0.44,0.00,0.77,0.82,0.93,0.60,0.50,0.95,0.80
8,Denmark,0.64,0.39,0.00,0.14,0.10,0.10,0.10,0.52,0.26,0.73
9,Ecuador,0.49,0.35,0.33,0.31,0.10,0.15,0.18,0.50,0.12,0.47


### Discipline axis

In [50]:
discipline_axis = team_df[['team','pens_conceded', 'fouls', 'fouled', 'cards_red', 'cards_yellow', 'pens_won']]
discipline_axis.describe()

,pens_conceded,fouls,fouled,cards_red,cards_yellow,pens_won
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,0.718750,49.968750,47.562500,0.125000,7.093750,0.593750
std,0.888434,19.924834,21.409882,0.336011,3.266244,0.979117
min,0.000000,24.000000,23.000000,0.000000,1.000000,0.000000
25%,0.000000,34.750000,34.750000,0.000000,5.000000,0.000000
50%,1.000000,46.500000,38.000000,0.000000,7.000000,0.000000
75%,1.000000,55.250000,51.750000,0.000000,8.000000,1.000000
max,4.000000,100.000000,115.000000,1.000000,17.000000,5.000000


In [51]:
lst5 = []
for i in discipline_axis.columns.copy():
    if 'pct' not in i and i != 'team':
        lst5.append(i)

print(lst5)
for team in lst5:
    min_max_scaling(discipline_axis, team)

['pens_conceded', 'fouls', 'fouled', 'cards_red', 'cards_yellow', 'pens_won']


/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)
/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingW

In [52]:
discipline_axis['discipline_score'] = round(0.05 * discipline_axis['pens_won'] +
                                            0.05 * discipline_axis['fouled'] -
                                            0.05 * discipline_axis['cards_yellow'] -
                                            0.08 * discipline_axis['cards_red'] - 
                                            0.06 * discipline_axis['fouls'] - 
                                            0.08 * discipline_axis['pens_conceded'],2)

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3185150110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  discipline_axis['discipline_score'] = round(0.05 * discipline_axis['pens_won'] +


In [53]:
discipline_axis

,team,pens_conceded,fouls,fouled,cards_red,cards_yellow,pens_won,discipline_score
0,Argentina,0.50,1.00,1.00,0.0,1.00,1.0,-0.05
1,Australia,0.00,0.37,0.12,0.0,0.38,0.0,-0.04
2,Belgium,0.25,0.08,0.13,0.0,0.25,0.0,-0.03
3,Brazil,0.00,0.51,0.55,0.0,0.31,0.2,-0.01
4,Cameroon,0.00,0.11,0.16,1.0,0.44,0.0,-0.10
5,Canada,0.00,0.12,0.15,0.0,0.44,0.0,-0.02
6,Costa Rica,0.25,0.00,0.15,0.0,0.31,0.0,-0.03
7,Croatia,0.25,0.87,0.73,0.0,0.44,0.0,-0.06
8,Denmark,0.00,0.05,0.00,0.0,0.25,0.0,-0.02
9,Ecuador,0.25,0.34,0.15,0.0,0.12,0.2,-0.03


In [54]:
min_max_scaling(discipline_axis, 'discipline_score')

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_60121/3692461358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = round((df[column_name] - min_value) / (max_value - min_value),2)


In [55]:
discipline_axis

,team,pens_conceded,fouls,fouled,cards_red,cards_yellow,pens_won,discipline_score
0,Argentina,0.50,1.00,1.00,0.0,1.00,1.0,0.67
1,Australia,0.00,0.37,0.12,0.0,0.38,0.0,0.72
2,Belgium,0.25,0.08,0.13,0.0,0.25,0.0,0.78
3,Brazil,0.00,0.51,0.55,0.0,0.31,0.2,0.89
4,Cameroon,0.00,0.11,0.16,1.0,0.44,0.0,0.39
5,Canada,0.00,0.12,0.15,0.0,0.44,0.0,0.83
6,Costa Rica,0.25,0.00,0.15,0.0,0.31,0.0,0.78
7,Croatia,0.25,0.87,0.73,0.0,0.44,0.0,0.61
8,Denmark,0.00,0.05,0.00,0.0,0.25,0.0,0.83
9,Ecuador,0.25,0.34,0.15,0.0,0.12,0.2,0.78


## Finalizing dataset for radar chart

In [56]:
countries = ['United States', 'Netherlands', 'Argentina', 
 'Australia', 'Japan', 'Croatia', 'Brazil', 'Korea Republic',
 'England', 'Senegal', 'France', 'Poland', 'Morocco', 'Spain', 
 'Portugal', 'Switzerland']

In [57]:
temp_radar1 = pd.merge(defensive_axis[['team', 'defensive_score']], attacking_axis[['team', 'attacking_score']], on = 'team')
temp_radar2 = pd.merge(possession_axis[['team', 'possession_score']], temp_radar1, on = 'team')
temp_radar3 = pd.merge(passes_axis[['team', 'passes_score']], temp_radar2, on = 'team')
radar_data = pd.merge(discipline_axis[['team', 'discipline_score']], temp_radar3, on = 'team')
radar_data = radar_data[radar_data['team'].isin(countries)].reset_index(drop = True)
radar_data

,team,discipline_score,passes_score,possession_score,defensive_score,attacking_score
0,Argentina,0.67,1.00,0.67,0.56,0.85
1,Australia,0.72,0.15,0.00,0.88,0.15
2,Brazil,0.89,0.89,0.47,0.92,0.65
3,Croatia,0.61,0.89,0.80,0.92,0.65
4,England,0.94,0.74,1.00,0.64,0.91
5,France,0.44,1.00,1.00,0.76,1.00
6,Japan,0.61,0.22,0.07,0.60,0.33
7,Korea Republic,0.67,0.37,0.53,0.52,0.43
8,Morocco,0.22,0.41,0.40,0.88,0.31
9,Netherlands,0.00,0.52,0.67,1.00,0.61


In [58]:
radar_data.sort_values(by = 'defensive_score', ascending = False).reset_index(drop = True)

,team,discipline_score,passes_score,possession_score,defensive_score,attacking_score
0,Netherlands,0.00,0.52,0.67,1.00,0.61
1,Brazil,0.89,0.89,0.47,0.92,0.65
2,Croatia,0.61,0.89,0.80,0.92,0.65
3,Poland,0.67,0.19,0.07,0.92,0.22
4,Australia,0.72,0.15,0.00,0.88,0.15
5,Morocco,0.22,0.41,0.40,0.88,0.31
6,France,0.44,1.00,1.00,0.76,1.00
7,United States,0.72,0.37,0.40,0.76,0.37
8,England,0.94,0.74,1.00,0.64,0.91
9,Spain,1.00,0.74,0.40,0.64,0.46


### Radar chart data saving

In [59]:
radar_data.to_json(r'/Users/ansat.omurzakov/Desktop/radar_data.json',orient = 'records')

# Match data

### Getting team results

In [60]:
def team_result(score):
    if '(' in score:
        if int(score[1]) > int(score[-2]):
            return 'win'
        elif int(score[-2]) > int(score[1]):
            return 'loss'
    else:
        if int(score[0]) > int(score[-1]):
            return 'win'
        elif int(score[-1]) > int(score[0]):
            return 'loss'
        else: 
            return 'tie'

def opposite_team_result(team_result):
    if team_result == 'win':
        return 'loss'
    elif team_result == 'tie':
        return 'tie'
    elif team_result == 'loss':
        return 'win'

In [61]:
match_df['home_team_result'] = match_df['score'].apply(team_result)
match_df['away_team_result'] = match_df['home_team_result'].apply(opposite_team_result)

In [62]:
match_df.columns

Index(['match', 'dayofweek', 'match_time', 'home_team', 'away_team', 'home_xg',
       'away_xg', 'score', 'attendance', 'venue', 'referee', 'home_formation',
       'away_formation', 'home_captain', 'away_captain', 'home_manager',
       'away_manager', 'home_possession', 'away_possession',
       'home_completed_passes', 'home_attempted_pases',
       'away_completed_passes', 'away_attempted_pases', 'home_sot', 'away_sot',
       'home_total_shots', 'away_total_shots', 'home_saves', 'away_saves',
       'home_fouls', 'away_fouls', 'home_corners', 'away_corners',
       'home_crosses', 'away_crosses', 'home_touches', 'away_touches',
       'home_tackles', 'away_tackles', 'home_interceptions',
       'away_interceptions', 'home_aerials_won', 'away_aerials_won',
       'home_clearances', 'away_clearances', 'home_offsides', 'away_offsides',
       'home_gks', 'away_gks', 'home_throw_ins', 'away_throw_ins',
       'home_long_balls', 'away_long_balls', 'home_team_result',
       'away_team

### Getting wins per strategy

In [63]:
temp = set(match_df['home_team'])
temp2 = set(match_df['away_team'])
countries = set(list(temp) + list(temp2))
countries = ['Iran' if i == 'IR Iran' else i for i in countries ]

In [64]:
# Home teams
home_results = match_df[['home_team', 'home_team_result']].groupby('home_team').value_counts()
countries_win = dict()
for i in countries:
    if i in list(match_df['home_team']) and ('win' in home_results[i].index):
        # print(i, home_results[i]['win'])
        countries_win[i] = home_results[i]['win']
    else:
        countries_win[i] = 0

# Away teams
away_results = match_df[['away_team', 'away_team_result']].groupby('away_team').value_counts()
for i in countries:
    if i in list(match_df['away_team']) and ('win' in away_results[i].index):
        countries_win[i] += away_results[i]['win']

In [65]:
# Home teams
home_results = match_df[['home_team', 'home_team_result']].groupby('home_team').value_counts()
countries_loss = dict()
for i in countries:
    if i in list(match_df['home_team']) and ('loss' in home_results[i].index):
        # print(i, home_results[i]['win'])
        countries_loss[i] = home_results[i]['loss']
    else:
        countries_loss[i] = 0

# Away teams
away_results = match_df[['away_team', 'away_team_result']].groupby('away_team').value_counts()
for i in countries:
    if i in list(match_df['away_team']) and ('loss' in away_results[i].index):
        countries_loss[i] += away_results[i]['loss']

In [66]:
# Home teams
home_results = match_df[['home_team', 'home_team_result']].groupby('home_team').value_counts()
countries_tie = dict()
for i in countries:
    if i in list(match_df['home_team']) and ('tie' in home_results[i].index):
        # print(i, home_results[i]['win'])
        countries_tie[i] = home_results[i]['tie']
    else:
        countries_tie[i] = 0

# Away teams
away_results = match_df[['away_team', 'away_team_result']].groupby('away_team').value_counts()
for i in countries:
    if i in list(match_df['away_team']) and ('tie' in away_results[i].index):
        countries_tie[i] += away_results[i]['tie']

In [67]:
wins = pd.DataFrame.from_dict(countries_win, orient = 'index', columns = ['win'])
ties = pd.DataFrame.from_dict(countries_tie, orient = 'index', columns = ['tie'])
loss = pd.DataFrame.from_dict(countries_loss, orient = 'index', columns = ['loss'])

In [68]:
wins['ties'] = ties
wins['loss'] = loss
wins = wins.reset_index()
wins.columns = ['team', 'win', 'ties', 'loss']

In [69]:
kiko = team_df[['team', 'Team Strategy']]

In [70]:
team_strat_wl = pd.merge(wins, kiko, on ='team')

In [71]:
countries = ['United States'.strip(), 'Netherlands', 'Argentina', 
 'Australia', 'Japan', 'Croatia', 'Brazil', 'Korea Republic',
 'England', 'Senegal', 'France', 'Poland', 'Morocco', 'Spain', 
 'Portugal', 'Switzerland']

In [72]:
team_strat_wl = team_strat_wl[team_strat_wl['team'].isin(countries)]

In [73]:
team_strat_wl = team_strat_wl.reset_index(drop = True)

In [74]:
team_strat_wl

,team,win,ties,loss,Team Strategy
0,Brazil,3,0,2,Bad attack; Good defense
1,England,3,1,1,Good attack; Good defense
2,Australia,2,0,2,Good attack; Bad defense
3,Senegal,2,0,2,Good attack; Bad defense
4,Japan,2,0,2,Good attack; Good defense
5,Spain,1,1,2,Good attack; Good defense
6,France,5,0,2,Good attack; Bad defense
7,Portugal,3,0,2,Good attack; Bad defense
8,Morocco,4,1,2,Bad attack; Good defense
9,Croatia,4,2,1,Good attack; Bad defense


### Team Strategy win/loss/tie saving

In [75]:
team_strat_wl.to_json(r'/Users/ansat.omurzakov/Desktop/sankey_data3.json',orient = 'records')

In [76]:
team_strat_wl.groupby('Team Strategy').sum('wins')

,win,ties,loss
Team Strategy,,,
Bad attack; Bad defense,2,0,2
Bad attack; Good defense,9,4,7
Good attack; Bad defense,23,3,12
Good attack; Good defense,9,3,6


# Historic data

In [77]:
df_historic_world_cup.sort_values(by = 'Attendance', ascending = False).head(10)

,Year,Host,Teams,Champion,Runner-Up,TopScorrer,Attendance,AttendanceAvg,Matches
7,1994,United States,24,Brazil,Italy,"Hristo Stoichkov, Oleg Salenko - 6",3587538,68991,52
2,2014,Brazil,32,Germany,Argentina,James Rodríguez - 6,3429873,53592,64
0,2022,Qatar,32,Argentina,France,Kylian Mbappé - 8,3404252,53191,64
4,2006,Germany,32,Italy,France,Miroslav Klose - 5,3352605,52384,64
3,2010,South Africa,32,Spain,Netherlands,"Wesley Sneijder, Thomas Müller... - 5",3178856,49670,64
1,2018,Russia,32,France,Croatia,Harry Kane - 6,3031768,47371,64
6,1998,France,32,France,Brazil,Davor Šuker - 6,2903477,45367,64
5,2002,"Korea Republic, Japan",32,Brazil,Germany,Ronaldo - 8,2705337,42271,64
8,1990,Italy,24,West Germany,Argentina,Salvatore Schillaci - 6,2516215,48389,52
9,1986,Mexico,24,Argentina,West Germany,Gary Lineker - 6,2394031,46039,52


In [78]:
countries = ['USA', 'Netherlands', 'Argentina', 
 'Australia', 'Japan', 'Croatia', 'Brazil', 'Korea Republic',
 'England', 'Senegal', 'France', 'Poland', 'Morocco', 'Spain', 
 'Portugal', 'Switzerland']

In [79]:
df_historic_fifa_ranking[df_historic_fifa_ranking['team'].isin(countries)].reset_index(drop = True)

,team,team_code,association,rank,previous_rank,points,previous_points
0,Brazil,BRA,CONMEBOL,1,1,1841.30,1837.56
1,Argentina,ARG,CONMEBOL,3,3,1773.88,1770.65
2,France,FRA,UEFA,4,4,1759.78,1764.85
3,England,ENG,UEFA,5,5,1728.47,1737.46
4,Spain,ESP,UEFA,7,6,1715.22,1716.93
5,Netherlands,NED,UEFA,8,8,1694.51,1679.41
6,Portugal,POR,UEFA,9,9,1676.56,1678.65
7,Croatia,CRO,UEFA,12,15,1645.64,1632.15
8,Switzerland,SUI,UEFA,15,16,1635.92,1621.43
9,USA,USA,CONCACAF,16,14,1627.48,1635.01


In [80]:
dick = {}
dick['ARG'] = 1855.2
dick['FRA'] = 1845.4
dick['ENG'] = 1800.5
dick['BRA'] = 1784.1
dick['ESP'] = 1732.6
dick['NED'] = 1745.5
dick['CRO'] = 1717.6
dick['POR'] = 1745.1
dick['SUI'] = 1613.4
dick['USA'] = 1665.3
dick['SEN'] = 1594.3
dick['MAR'] = 1661.7
dick['JPN'] = 1620.2
dick['POL'] = 1520.2
dick['AUS'] = 1539.2
dick['KOR'] = 1550.6

In [81]:
ranking = pd.DataFrame.from_dict(dick, orient='index').reset_index()
ranking.columns = ['team_code', 'current_points']
rankings = pd.merge(df_historic_fifa_ranking, ranking, on = 'team_code')
rankings

,team,team_code,association,rank,previous_rank,points,previous_points,current_points
0,Brazil,BRA,CONMEBOL,1,1,1841.30,1837.56,1784.1
1,Argentina,ARG,CONMEBOL,3,3,1773.88,1770.65,1855.2
2,France,FRA,UEFA,4,4,1759.78,1764.85,1845.4
3,England,ENG,UEFA,5,5,1728.47,1737.46,1800.5
4,Spain,ESP,UEFA,7,6,1715.22,1716.93,1732.6
5,Netherlands,NED,UEFA,8,8,1694.51,1679.41,1745.5
6,Portugal,POR,UEFA,9,9,1676.56,1678.65,1745.1
7,Croatia,CRO,UEFA,12,15,1645.64,1632.15,1717.6
8,Switzerland,SUI,UEFA,15,16,1635.92,1621.43,1613.4
9,USA,USA,CONCACAF,16,14,1627.48,1635.01,1665.3


In [82]:
rankings.loc[9, 'team'] = 'United States'

In [83]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [84]:
countries = ['United States'.strip(), 'Netherlands', 'Argentina', 
 'Australia', 'Japan', 'Croatia', 'Brazil', 'Korea Republic',
 'England', 'Senegal', 'France', 'Poland', 'Morocco', 'Spain', 
 'Portugal', 'Switzerland']
goals_per_team = team_df[team_df['team'].isin(countries)][['team','goals']].reset_index(drop = True)

In [85]:
goals_per_team

,team,goals
0,Argentina,15
1,Australia,3
2,Brazil,8
3,Croatia,8
4,England,13
5,France,16
6,Japan,5
7,Korea Republic,5
8,Morocco,6
9,Netherlands,10


In [86]:
goals_per_team['team'] = goals_per_team['team'].apply(lambda x: x.strip())

In [87]:
rankings = pd.merge(rankings, goals_per_team, on = 'team')

In [88]:
team_strat_wl

,team,win,ties,loss,Team Strategy
0,Brazil,3,0,2,Bad attack; Good defense
1,England,3,1,1,Good attack; Good defense
2,Australia,2,0,2,Good attack; Bad defense
3,Senegal,2,0,2,Good attack; Bad defense
4,Japan,2,0,2,Good attack; Good defense
5,Spain,1,1,2,Good attack; Good defense
6,France,5,0,2,Good attack; Bad defense
7,Portugal,3,0,2,Good attack; Bad defense
8,Morocco,4,1,2,Bad attack; Good defense
9,Croatia,4,2,1,Good attack; Bad defense


In [89]:
final_ranking = rankings[['team', 'team_code', 'points', 'previous_points', 'current_points', 'goals']]
final_ranking.columns = ['team', 'team_code', 'points_2018', 'points_2014', 'points_2022',
       'goals']
final_ranking

,team,team_code,points_2018,points_2014,points_2022,goals
0,Brazil,BRA,1841.30,1837.56,1784.1,8
1,Argentina,ARG,1773.88,1770.65,1855.2,15
2,France,FRA,1759.78,1764.85,1845.4,16
3,England,ENG,1728.47,1737.46,1800.5,13
4,Spain,ESP,1715.22,1716.93,1732.6,9
5,Netherlands,NED,1694.51,1679.41,1745.5,10
6,Portugal,POR,1676.56,1678.65,1745.1,12
7,Croatia,CRO,1645.64,1632.15,1717.6,8
8,Switzerland,SUI,1635.92,1621.43,1613.4,5
9,United States,USA,1627.48,1635.01,1665.3,3


In [90]:
final_ranking = pd.merge(final_ranking, team_strat_wl[['team', 'Team Strategy']], on = 'team')
final_ranking

,team,team_code,points_2018,points_2014,points_2022,goals,Team Strategy
0,Brazil,BRA,1841.30,1837.56,1784.1,8,Bad attack; Good defense
1,Argentina,ARG,1773.88,1770.65,1855.2,15,Good attack; Bad defense
2,France,FRA,1759.78,1764.85,1845.4,16,Good attack; Bad defense
3,England,ENG,1728.47,1737.46,1800.5,13,Good attack; Good defense
4,Spain,ESP,1715.22,1716.93,1732.6,9,Good attack; Good defense
5,Netherlands,NED,1694.51,1679.41,1745.5,10,Good attack; Good defense
6,Portugal,POR,1676.56,1678.65,1745.1,12,Good attack; Bad defense
7,Croatia,CRO,1645.64,1632.15,1717.6,8,Good attack; Bad defense
8,Switzerland,SUI,1635.92,1621.43,1613.4,5,Bad attack; Bad defense
9,United States,USA,1627.48,1635.01,1665.3,3,Bad attack; Good defense


### Team ranking saving

In [91]:
# final_ranking.to_json(r'/Users/ansat.omurzakov/Desktop/team_ranking.json', orient = 'records')

In [92]:
# fig, ax = plt.subplots(ncols = 1, nrows = 1, figsize = (20, 10))
px.scatter(final_ranking, x = ['points_2022'], y = 'goals', color = 'team', hover_data = ['Team Strategy'],
labels = {'value': 'Amount of points', 'goals': '# of goals' }, title = 'Goal to points comparison')